# "Segmentez des clients d'un site e-commerce"
_Cleaning Notebook_

## 0 Preliminaries

In [700]:
# !pip install flake8 pycodestyle_magic
# !pip install gtts

In [701]:
# !pip freeze > requirements_cleaning_eda.txt

Checking for PEP8 compliance

In [702]:
# %load_ext pycodestyle_magic
# %pycodestyle_on
# %pycodestyle_off

### 0.0 Importing Packages and Modules

Checking whether the notebook is on Colab or PC

In [703]:
import sys
is_colab = 'google.colab' in sys.modules
is_colab, sys.executable

(True, '/usr/bin/python3')

Mounting my Drive if on Colab

In [704]:
if is_colab==True:
    from google.colab import files, output, drive
    drive.mount('/gdrive')
    %cd /gdrive
    print("You're on Google Colab")
else:
    print("You're on a PC")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
You're on Google Colab


Installations and importations required in the virtual environment.

In [705]:
import os
if is_colab==True:
    if os.getcwd()!='/gdrive/My Drive/--DATA SCIENCE/PROJET5/NOTEBOOKS':
        os.chdir('My Drive/--DATA SCIENCE/PROJET5/NOTEBOOKS')
else:
    if not (os.path.exists(os.getcwd()+'/requirements_cleaning_eda.txt') \
                     and os.path.exists(os.getcwd()+'/P5_functions.py')):
        print("ERROR: Make sure 'P5_functions.py' and \
'requirements_cleaning_eda.txt' are in the current working directory")

!pip install -r requirements_cleaning_eda.txt
from P5_functions import *

Importation of modules and packages. 

In [706]:
import io

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor']='w'

# import warnings
# warnings.filterwarnings('ignore')

import missingno as msno
# import ppscore as pps

import dill

Setting pandas display options.

In [707]:
dictPdSettings = {'display.max_rows': 500, 'display.width': 100,
                    'display.max_colwidth': 100,
                    'display.float_format': lambda x: '%.2f' % x}
for k,v in dictPdSettings.items():
  pd.set_option(k,v)

To play audio text-to-speech during execution.

In [708]:
from IPython.display import Audio
from gtts import gTTS

def speak(text, lang='en'):
    with io.BytesIO() as f:
        gTTS(text=text, lang=lang).write_to_fp(f)
        f.seek(0)
        return Audio(f.read(), autoplay=True)

In [709]:
speak('Packages and modules successfully imported')

### 0.1 Importing the datasets

Data is composed of 9 distinct .csv files we'll load in a dictionnary of dataframes.

In [710]:
paths = {'cust': '../DATA/olist_customers_dataset.csv' ,
         'geoloc':  '../DATA/olist_geolocation_dataset.csv' ,
         'ord_it':  '../DATA/olist_order_items_dataset.csv' ,
         'ord_pay':  '../DATA/olist_order_payments_dataset.csv' ,
         'ord_rev':  '../DATA/olist_order_reviews_dataset.csv' ,
         'orders':  '../DATA/olist_orders_dataset.csv' ,
         'prod':  '../DATA/olist_products_dataset.csv' ,
         'sell':  '../DATA/olist_sellers_dataset.csv' ,
         'transl':  '../DATA/product_category_name_translation.csv' }

if is_colab==True:
    # Importing database from my Drive
    print("Try to import data files in the notebook from myDrive...")
else:
    # Importing database from PC
    print("Try to import data files in the notebook from PC ('DATA')...")

for k,v in paths.items():
    dfs = {k: pd.read_csv(v, sep=',',  encoding ='utf-8') for k,v in paths.items()}

print("-----> Importation of .csv in the notebook: OK")

Try to import data files in the notebook from myDrive...
-----> Importation of .csv in the notebook: OK


All the dates in the dataframes are to be converted to datetime.

In [711]:
for name_df in dfs.keys():
    date_cols = list(dfs[name_df]\
                     .columns[dfs[name_df].columns.str.contains('date|_at|time',
                                                                regex=True)])
    print("----Dates columns in df['{}']: ".format(name_df), date_cols)
    dfs[name_df][date_cols] = dfs[name_df][date_cols].apply(pd.to_datetime)
    # print(dfs[name_df].columns)

----Dates columns in df['cust']:  []
----Dates columns in df['geoloc']:  []
----Dates columns in df['ord_it']:  ['shipping_limit_date']
----Dates columns in df['ord_pay']:  []
----Dates columns in df['ord_rev']:  ['review_creation_date', 'review_answer_timestamp']
----Dates columns in df['orders']:  ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
----Dates columns in df['prod']:  []
----Dates columns in df['sell']:  []
----Dates columns in df['transl']:  []


In [712]:
speak('Datasets successfully imported')

### 0.2 First Overview

#### General description

In [713]:
for k,v in dfs.items():
    print(f'___________________{k.upper()}___________________')
    display(v.describe(include='all'))

___________________CUST___________________


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
count,99441,99441,99441.00,99441,99441
unique,99441,96096,nan,4119,27
top,560c13ab2e56044eec5b75a29e759272,8d50f5eadf50201ccdcedfb9e2ac8455,nan,sao paulo,SP
freq,1,17,nan,15540,41746
mean,NaN,NaN,35137.47,NaN,NaN
std,NaN,NaN,29797.94,NaN,NaN
min,NaN,NaN,1003.00,NaN,NaN
25%,NaN,NaN,11347.00,NaN,NaN
50%,NaN,NaN,24416.00,NaN,NaN
75%,NaN,NaN,58900.00,NaN,NaN


___________________GEOLOC___________________


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
count,1000163.00,1000163.00,1000163.00,1000163,1000163
unique,nan,nan,nan,8011,27
top,nan,nan,nan,sao paulo,SP
freq,nan,nan,nan,135800,404268
mean,36574.17,-21.18,-46.39,NaN,NaN
std,30549.34,5.72,4.27,NaN,NaN
min,1001.00,-36.61,-101.47,NaN,NaN
25%,11075.00,-23.60,-48.57,NaN,NaN
50%,26530.00,-22.92,-46.64,NaN,NaN
75%,63504.00,-19.98,-43.77,NaN,NaN


___________________ORD_IT___________________


...

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32951,32341,32341.00,32341.00,32341.00,32949.00,32949.00,32949.00,32949.00
unique,32951,73,nan,nan,nan,nan,nan,nan,nan
top,04566e1a1afa83470b25c2642706fc32,cama_mesa_banho,nan,nan,nan,nan,nan,nan,nan
freq,1,3029,nan,nan,nan,nan,nan,nan,nan
mean,NaN,NaN,48.48,771.50,2.19,2276.47,30.82,16.94,23.20
std,NaN,NaN,10.25,635.12,1.74,4282.04,16.91,13.64,12.08
min,NaN,NaN,5.00,4.00,1.00,0.00,7.00,2.00,6.00
25%,NaN,NaN,42.00,339.00,1.00,300.00,18.00,8.00,15.00
50%,NaN,NaN,51.00,595.00,1.00,700.00,25.00,13.00,20.00
75%,NaN,NaN,57.00,972.00,3.00,1900.00,38.00,21.00,30.00


___________________SELL___________________


,seller_id,seller_zip_code_prefix,seller_city,seller_state
count,3095,3095.00,3095,3095
unique,3095,nan,611,23
top,8b87cd0d43d49b9bdf40f56229364d59,nan,sao paulo,SP
freq,1,nan,694,1849
mean,NaN,32291.06,NaN,NaN
std,NaN,32713.45,NaN,NaN
min,NaN,1001.00,NaN,NaN
25%,NaN,7093.50,NaN,NaN
50%,NaN,14940.00,NaN,NaN
75%,NaN,64552.50,NaN,NaN


___________________TRANSL___________________


,product_category_name,product_category_name_english
count,71,71
unique,71,71
top,musica,construction_tools_lights
freq,1,1


Printing total nb and percentage of null:

In [714]:
for k,v in dfs.items():
    print(f'___________________{k.upper()}___________________')
    display(print_null_pct(v))

___________________CUST___________________
nb of null:  0 
pct of null:  0.0


None

___________________GEOLOC___________________
nb of null:  0 
pct of null:  0.0


None

___________________ORD_IT___________________
nb of null:  0 
pct of null:  0.0


...

None

___________________SELL___________________
nb of null:  0 
pct of null:  0.0


None

___________________TRANSL___________________
nb of null:  0 
pct of null:  0.0


None

### 0.3 Merging and aggregating the 'orders' databases

#### Geolocation

In [715]:
dfs['geoloc'].columns

Index(['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city',
       'geolocation_state'],
      dtype='object')

The geolocation dataset provides the city, state, latitude and longitude corresponding to Brazil zipcodes.

We notice that this data could be used to calculate the geographical distance between sellers and vendor. However we won't use this distance in our segmentation of clients.

In [716]:
print("States in Brazil (Geolocation dataset) ({}):\n{}"\
      .format(dfs['geoloc'].geolocation_state.nunique(),
              dfs['geoloc'].geolocation_state.unique()))
print("Cities in Brazil (Geolocation dataset): {}"\
      .format(dfs['geoloc'].geolocation_city.nunique()))
print("ZipCodes in Brazil (Geolocation dataset): {}"\
      .format(dfs['geoloc'].geolocation_zip_code_prefix.nunique()))

States in Brazil (Geolocation dataset) (27):
['SP' 'RN' 'AC' 'RJ' 'ES' 'MG' 'BA' 'SE' 'PE' 'AL' 'PB' 'CE' 'PI' 'MA'
 'PA' 'AP' 'AM' 'RR' 'DF' 'GO' 'RO' 'TO' 'MT' 'MS' 'RS' 'PR' 'SC']
Cities in Brazil (Geolocation dataset): 8011
ZipCodes in Brazil (Geolocation dataset): 19015


The number of different states, cities and zipcodes are far more important to keep the corresponding columns.

We'll group states in 5 regions.

In [717]:
braz_reg = pd.read_csv('Brazil_regions.csv', header=None, encoding='latin-1',
                       sep=';')
braz_reg = braz_reg.rename(columns={0: 'region',
                                    1: 'name_region',
                                    2: 'code_region'})
display(braz_reg.head(3))

,region,name_region,code_region
0,N,Acre,AC
1,NE,Alagoas,AL
2,N,Amapá,AP


In [718]:
dict_reg = dict(braz_reg.groupby('region')\
                .apply(lambda x: tuple(x['code_region'])))
dict_reg

{'CW': ('DF', 'GO', 'MT', 'MS'),
 'N': ('AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'),
 'NE': ('AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'),
 'S': ('PR', 'RS', 'SC'),
 'SE': ('ES', 'MG', 'RJ', 'SP')}

In [719]:
df1 = dfs['geoloc']
df2 = braz_reg[['region','code_region']]

dfs['geoloc'] = pd.merge(df1, df2, 'left',
                         left_on='geolocation_state', right_on='code_region')
dfs['geoloc'] = dfs['geoloc'].drop(columns=['code_region',
                                            # 'geolocation_lat',
                                            # 'geolocation_lng',
                                            'geolocation_city', 
                                            # 'geolocation_zip_code_prefix',
                                            ])

In [720]:
dfs['geoloc'].sample(3)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_state,region
294756,13142,-22.77,-47.17,SP,SE
624985,37150,-20.98,-46.12,MG,SE
392594,18680,-22.59,-48.80,SP,SE


#### Sellers

In [721]:
dfs['sell'].columns

Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')

The Sellers dataset gives information about their localisation.
Three levels of precision are given : 
- zipcode
- city
- state

In [722]:
print("Unique seller: ", dfs['sell'].seller_id.nunique())
print("Shape of sellers dataset: ", dfs['sell'].shape)

Unique seller:  3095
Shape of sellers dataset:  (3095, 4)


In [723]:
print("States of sellers (Sellers dataset) ({}):\n{}"\
      .format(dfs['sell'].seller_state.nunique(),
              dfs['sell'].seller_state.unique()))
print("Cities of sellers (Sellers dataset): {}"\
      .format(dfs['sell'].seller_city.nunique()))
print("Zipcodes of sellers (Sellers dataset): {}"\
      .format(dfs['sell'].seller_zip_code_prefix.nunique()))

States of sellers (Sellers dataset) (23):
['SP' 'RJ' 'PE' 'PR' 'GO' 'SC' 'BA' 'DF' 'RS' 'MG' 'RN' 'MT' 'CE' 'PB'
 'AC' 'ES' 'RO' 'PI' 'MS' 'SE' 'MA' 'AM' 'PA']
Cities of sellers (Sellers dataset): 611
Zipcodes of sellers (Sellers dataset): 2246


Removing duplicates in the geolocation database:

In [724]:
dfs['geoloc'] = dfs['geoloc'].groupby("geolocation_zip_code_prefix")\
                             .max().reset_index()

In [725]:
dfs['geoloc'].shape

(19015, 5)

Checking for duplicates in the sellers database:

In [726]:
dfs['sell'].groupby('seller_id').size().sum() ==\
    dfs['sell'].shape[0]

True

Merging geolocalisation data with sellers data:

In [727]:
dfs['sell'] = pd.merge(dfs['sell'], dfs['geoloc'], how='left',
                       left_on='seller_zip_code_prefix',
                       right_on='geolocation_zip_code_prefix')\
                            .drop(columns=['seller_state', 'seller_city',
                                           'geolocation_zip_code_prefix',
                                           'seller_zip_code_prefix'])\
                            .rename(columns={'region': 'seller_region',
                                             'geolocation_lat': 'seller_lat',
                                             'geolocation_lng': 'seller_long'})

In [728]:
dfs['sell'].sample(3)

,seller_id,seller_lat,seller_long,geolocation_state,seller_region
611,fe87f472055fbcf1d7e691c00b1560dc,-22.74,-47.05,SP,SE
593,30829ded4523ab9224b93bc49a62c95f,-28.59,-51.77,RS,S
1889,1de62b6f2fd96227629786db492433db,-21.22,-50.45,SP,SE


#### Categories

Products are classified in categories. A translation file is provided in order to get the english names of product categories.

In [729]:
dfs['transl'].columns, dfs['transl'].shape, dfs['transl'].nunique()

(Index(['product_category_name', 'product_category_name_english'], dtype='object'),
 (71, 2),
 product_category_name            71
 product_category_name_english    71
 dtype: int64)

Let's have a look to the 71 categories:

In [730]:
dfs['transl'].iloc[:,1].unique()

array(['health_beauty', 'computers_accessories', 'auto', 'bed_bath_table',
       'furniture_decor', 'sports_leisure', 'perfumery', 'housewares',
       'telephony', 'watches_gifts', 'food_drink', 'baby', 'stationery',
       'tablets_printing_image', 'toys', 'fixed_telephony',
       'garden_tools', 'fashion_bags_accessories', 'small_appliances',
       'consoles_games', 'audio', 'fashion_shoes', 'cool_stuff',
       'luggage_accessories', 'air_conditioning',
       'construction_tools_construction',
       'kitchen_dining_laundry_garden_furniture',
       'costruction_tools_garden', 'fashion_male_clothing', 'pet_shop',
       'office_furniture', 'market_place', 'electronics',
       'home_appliances', 'party_supplies', 'home_confort',
       'costruction_tools_tools', 'agro_industry_and_commerce',
       'furniture_mattress_and_upholstery', 'books_technical',
       'home_construction', 'musical_instruments',
       'furniture_living_room', 'construction_tools_lights',
       'indust

Adding lines for missing translations:

- ('unknown': 'unknown'),
- ('beleza_saude': 'health_beauty'),
- ('pc_gamer': 'pc_gamer'),
- ('portateis_cozinha_e_preparadores_de_alimentos': 'cooking tools')

In [731]:
cols = dfs['transl'].columns

df_app = \
    pd.DataFrame({'product_category_name':
                  ['beleza_saude', 'pc_gamer',
                   'portateis_cozinha_e_preparadores_de_alimentos', 'unknown'],
                  'product_category_name_english':
                  ['health_beauty', 'pc_gamer', 'kitchen_utensils', 'unknown']})
    
dfs['transl'] = dfs['transl'].append(df_app, ignore_index=True)

These categories can be splitted into only 13 categories. We will allow categories to belong to 2 or more bigger categories at the same time.

In order to prepare the aggreagation of the order_item dataset, each category is represented as a column.

In [732]:
# Dictionary of the keywords in categories related to bigger categories
big_cat_dict = {
                'cat_electronics' : ['telephony', 'watches', 'consoles',
                                     'computer', 'tablets', 'appliance',
                                     'electronics'],
                'cat_furniture' : ['furniture', 'table', 'bed'],
                'cat_food' : ['food', 'drink', 'cuisine'],
                'cat_home' : ['home', 'house', 'kitchen', 'flowers', 'bed',
                              'decor', 'conditioning', 'party', 'christmas',
                              'luggage'],
                'cat_diy' : ['struction', 'tools', 'craft'],
                'cat_fashion' : ['fash'],
                'cat_leisure' : ['garden', 'toys', 'game', 'cool', 'games',
                                 'art', 'sports', 'music', 'cine', 'beach',
                                 'pc'],
                'cat_multimedia' : ['dvd', 'audio'],
                'cat_books' : ['books'],
                'cat_hygiene' : ['health', 'perfumery', 'baby', 'hygiene' ],
                'cat_office' : ['office', 'stationery', 'business'],
                'cat_miscellaneous' :  ['appliances', 'security', 'auto',
                                        'pet', 'market', 'industry'],
                'cat_unknown': ['unknown']
                }

# Create new dummy columns for bigger categories
for big_cat, li_words in big_cat_dict.items():
    dfs['transl'][big_cat] = dfs['transl'].iloc[:,1]\
        .str.contains('|'.join(li_words), regex=True).map({False: 0, True: 1})

dfs['transl'].sample(5)

,product_category_name,product_category_name_english,cat_electronics,cat_furniture,cat_food,cat_home,cat_diy,cat_fashion,cat_leisure,cat_multimedia,cat_books,cat_hygiene,cat_office,cat_miscellaneous,cat_unknown
19,consoles_games,consoles_games,1,0,0,0,0,0,1,0,0,0,0,0,0
47,moveis_quarto,furniture_bedroom,0,1,0,1,0,0,0,0,0,0,0,0,0
35,casa_conforto,home_confort,0,0,0,1,0,0,0,0,0,0,0,0,0
68,fraldas_higiene,diapers_and_hygiene,0,0,0,0,0,0,0,0,0,1,0,0,0
23,malas_acessorios,luggage_accessories,0,0,0,1,0,0,0,0,0,0,0,0,0


#### Products / Items

In [733]:
dfs['prod'].columns

Index(['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm',
       'product_width_cm'],
      dtype='object')

Let's see if there are missing data.

In [734]:
dfs['prod'].isna().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

We'll fill missing values in product_category_name with 'unknown', and product_description_length' with 0.

In [735]:
dfs['prod'] = dfs['prod'].fillna(value ={'product_category_name': 'unknown',
                                         'product_description_lenght': 0})

In [736]:
dfs['prod'].isna().sum()

product_id                      0
product_category_name           0
product_name_lenght           610
product_description_lenght      0
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

We'll use the translation file (as a dictionary) to get the english names of categories and create the bigger categories columns.

In [737]:
cat_dict = dfs['transl'].set_index('product_category_name')\
    .to_dict()['product_category_name_english']

In [738]:
dfs['prod'].product_category_name = \
    dfs['prod'].product_category_name.map(cat_dict)

In [739]:
dfs['prod'].product_category_name.unique()

array(['perfumery', 'art', 'sports_leisure', 'baby', 'housewares',
       'musical_instruments', 'cool_stuff', 'furniture_decor',
       'home_appliances', 'toys', 'bed_bath_table',
       'construction_tools_safety', 'computers_accessories',
       'health_beauty', 'luggage_accessories', 'garden_tools',
       'office_furniture', 'auto', 'electronics', 'fashion_shoes',
       'telephony', 'stationery', 'fashion_bags_accessories', 'computers',
       'home_construction', 'watches_gifts',
       'construction_tools_construction', 'pet_shop', 'small_appliances',
       'agro_industry_and_commerce', 'unknown', 'furniture_living_room',
       'signaling_and_security', 'air_conditioning', 'consoles_games',
       'books_general_interest', 'costruction_tools_tools',
       'fashion_underwear_beach', 'fashion_male_clothing',
       'kitchen_dining_laundry_garden_furniture',
       'industry_commerce_and_business', 'fixed_telephony',
       'construction_tools_lights', 'books_technical',
     

In [740]:
for big_cat, li_words in big_cat_dict.items():
    dfs['prod'][big_cat] = dfs['prod'].product_category_name\
        .str.contains('|'.join(li_words), regex=True).map({False: 0, True: 1})

In [741]:
dfs['prod'].head(3)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,cat_electronics,cat_furniture,cat_food,cat_home,cat_diy,cat_fashion,cat_leisure,cat_multimedia,cat_books,cat_hygiene,cat_office,cat_miscellaneous,cat_unknown
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.00,287.00,1.00,225.00,16.00,10.00,14.00,0,0,0,0,0,0,0,0,0,1,0,0,0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.00,276.00,1.00,1000.00,30.00,18.00,20.00,0,0,0,0,0,0,1,0,0,0,0,0,0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.00,250.00,1.00,154.00,18.00,9.00,15.00,0,0,0,0,0,0,1,0,0,0,0,0,0


We will create a volume product column, from the length, height and width.

In [742]:
dfs['prod']['product_volume_cm3'] =\
    dfs['prod'].product_length_cm *\
    dfs['prod'].product_height_cm *\
    dfs['prod'].product_width_cm

The drop the 'product_photos_qty' and the 'product_name_lenght' columns, as well as length, heigth and width.

In [743]:
dfs['prod'] = dfs['prod'].drop(columns=['product_name_lenght',
                                        'product_photos_qty',
                                        'product_length_cm',
                                        'product_height_cm',
                                        'product_width_cm'])

In [744]:
dfs['prod'].dtypes

product_id                     object
product_category_name          object
product_description_lenght    float64
product_weight_g              float64
cat_electronics                 int64
cat_furniture                   int64
cat_food                        int64
cat_home                        int64
cat_diy                         int64
cat_fashion                     int64
cat_leisure                     int64
cat_multimedia                  int64
cat_books                       int64
cat_hygiene                     int64
cat_office                      int64
cat_miscellaneous               int64
cat_unknown                     int64
product_volume_cm3            float64
dtype: object

Missing weights and volumes are imputed with the median value:

In [745]:
median_vol = np.nanmedian(dfs['prod']['product_volume_cm3'])
dfs['prod']['product_volume_cm3'] = \
    dfs['prod']['product_volume_cm3'].fillna(median_vol)

median_wei = np.nanmedian(dfs['prod']['product_weight_g'])
dfs['prod']['product_weight_g'] = \
    dfs['prod']['product_weight_g'].fillna(median_wei)

In [746]:
median_vol, median_wei

(6840.0, 700.0)

In [747]:
dfs['prod'].isna().sum()

product_id                    0
product_category_name         0
product_description_lenght    0
product_weight_g              0
cat_electronics               0
cat_furniture                 0
cat_food                      0
cat_home                      0
cat_diy                       0
cat_fashion                   0
cat_leisure                   0
cat_multimedia                0
cat_books                     0
cat_hygiene                   0
cat_office                    0
cat_miscellaneous             0
cat_unknown                   0
product_volume_cm3            0
dtype: int64

#### Order items

In [748]:
dfs['ord_it'].columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price',
       'freight_value'],
      dtype='object')

In [749]:
dfs['orders'].columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date'],
      dtype='object')

(((We are not interested in the sellers id. We drop the column.)))

In [750]:
# dfs['ord_it'] = dfs['ord_it'].drop(columns=['seller_id'])

Let's merge the product database with the orders database. We choose a left join because we only want to keep the list of items that has been ordered.

We'll join on the 'product_id' column on the left (dfs['ord_it']) and on the right (dfs['prod']).

In [751]:
dfs['ord_it'] = pd.merge(dfs['ord_it'], dfs['prod'], 'left', on='product_id')

In [752]:
print("Available columns in 'order_items' dataset", dfs['ord_it'].columns)

Available columns in 'order_items' dataset Index(['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price',
       'freight_value', 'product_category_name', 'product_description_lenght', 'product_weight_g',
       'cat_electronics', 'cat_furniture', 'cat_food', 'cat_home', 'cat_diy', 'cat_fashion',
       'cat_leisure', 'cat_multimedia', 'cat_books', 'cat_hygiene', 'cat_office',
       'cat_miscellaneous', 'cat_unknown', 'product_volume_cm3'],
      dtype='object')


We want the category dummy columns to be proportional to the price of the item bought:

NB: there can be more than one "hot" category column for an item (see above the method used to reduce the 71 categories to 13).

In [753]:
cat_cols = dfs['ord_it'].columns[dfs['ord_it'].columns\
                                 .str.contains('cat_')].to_list()

In [754]:
for c in cat_cols:
    dfs['ord_it'][c] = dfs['ord_it'][c] * dfs['ord_it']['price']

In [755]:
dfs['ord_it'].sample(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_description_lenght,product_weight_g,cat_electronics,cat_furniture,cat_food,cat_home,cat_diy,cat_fashion,cat_leisure,cat_multimedia,cat_books,cat_hygiene,cat_office,cat_miscellaneous,cat_unknown,product_volume_cm3
72434,a532c4a850449b1f08cbc448e2a2568f,1,c403e106353e1aa40efe783ecb39ed7a,d2374cbcbb3ca4ab1086534108cc3ab7,2017-04-28 03:15:35,38.90,10.96,bed_bath_table,169.00,550.00,0.00,38.90,0.00,38.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6000.00
91413,cf6af03cfe2aa7ce2b23c2b77c1ff722,1,6e835aea84ae8eb68b8c14878dd43b30,4d6d651bd7684af3fffabd5f08d12e5a,2018-07-04 17:18:12,69.90,26.79,sports_leisure,831.00,2350.00,0.00,0.00,0.00,0.00,0.00,0.00,69.90,0.00,0.00,0.00,0.00,0.00,0.00,8000.00
1561,039ab3369f399f5a3ac25e99435a8251,1,16bf176650a888512655cc94f61860e3,6560211a19b47992c3666cc44a7e94c0,2018-06-26 02:58:52,39.00,50.99,watches_gifts,401.00,200.00,39.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,352.00


The result id a dataset providing for each line (item) the product bought, and the order in which it has been bought.

In [756]:
print("Total number of products bought: ", dfs['ord_it'].shape[0])
print("Total number of distinct orders: ", dfs['ord_it'].order_id.nunique())

Total number of products bought:  112650
Total number of distinct orders:  98666


Each order may contain one or several items.

Let's have a look on the orders containing more than one item:

In [757]:
dfs['ord_it'][dfs['ord_it'].duplicated('order_id', keep=False)]\
    .sort_values(['order_id', 'order_item_id']).iloc[159:166]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_description_lenght,product_weight_g,cat_electronics,cat_furniture,cat_food,cat_home,cat_diy,cat_fashion,cat_leisure,cat_multimedia,cat_books,cat_hygiene,cat_office,cat_miscellaneous,cat_unknown,product_volume_cm3
725,01b06b4d63c340902dbe3d9f8b2fe821,1,a659cb33082b851fb87a33af8f0fff29,817245bcc3badd82bbd222e0366951a6,2018-06-12 11:54:52,84.90,13.03,auto,380.00,150.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,84.90,0.00,1056.00
726,01b06b4d63c340902dbe3d9f8b2fe821,2,a659cb33082b851fb87a33af8f0fff29,817245bcc3badd82bbd222e0366951a6,2018-06-12 11:54:52,84.90,13.03,auto,380.00,150.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,84.90,0.00,1056.00
729,01b1a7fdae9ad1837d6ab861705a1fa5,1,3fae92f8d0ebb3317991934a6d717c47,1900267e848ceeba8fa32d80c1a5f5a8,2017-06-02 16:02:58,49.99,16.17,bed_bath_table,306.00,1250.00,0.00,49.99,0.00,49.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2560.00
730,01b1a7fdae9ad1837d6ab861705a1fa5,2,9b02b650be0a39d8590be7d35be443f9,ad273c9eb54ecb4532de4bf75bae9e4e,2017-06-02 16:02:58,59.00,10.68,housewares,660.00,1200.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7500.00
746,01ba82170ebb683c73793bb809bbf1b9,1,d6fe3b4ddecd4a8393c6a1385de3bfb6,7c67e1448b00f6e969d365cea6b010ab,2018-02-20 15:11:35,192.97,33.59,office_furniture,1157.00,16650.00,0.00,192.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,192.97,0.00,0.00,87696.00
747,01ba82170ebb683c73793bb809bbf1b9,2,d6fe3b4ddecd4a8393c6a1385de3bfb6,7c67e1448b00f6e969d365cea6b010ab,2018-02-20 15:11:35,192.97,33.59,office_furniture,1157.00,16650.00,0.00,192.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,192.97,0.00,0.00,87696.00
748,01ba82170ebb683c73793bb809bbf1b9,3,d6fe3b4ddecd4a8393c6a1385de3bfb6,7c67e1448b00f6e969d365cea6b010ab,2018-02-20 15:11:35,192.97,33.59,office_furniture,1157.00,16650.00,0.00,192.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,192.97,0.00,0.00,87696.00


Most of the multi-items orders contain the same item (see idexes 725 and 726), but it is sometimes not the case (729 and 730 for example).

Let's aggregate all the lines of dfs['ord_it'] correponding to the same order to get a dataframe with one line per order.
For each group of line with the same 'order_id', we'll:
- take max of 'order_item_id' (to have the total number of items in the order)
- take the mean of the product_description_length
- take sum of the number of one_hot columns that indicate the category
- take sum of price and sum of freight_value
- take the max of shipping_limit_date (to have the last date at which all items are supposedly received)
- take the most frequent category of the order

Then we'll drop the 'product_id' column.

In [758]:
dfs['ord_it'].columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price',
       'freight_value', 'product_category_name', 'product_description_lenght', 'product_weight_g',
       'cat_electronics', 'cat_furniture', 'cat_food', 'cat_home', 'cat_diy', 'cat_fashion',
       'cat_leisure', 'cat_multimedia', 'cat_books', 'cat_hygiene', 'cat_office',
       'cat_miscellaneous', 'cat_unknown', 'product_volume_cm3'],
      dtype='object')

In [759]:
def most_freq_cat(x): return x if x.size == 1 else x.value_counts().idxmax()

agg_dict_1 = {'order_item_id': np.max,
              'seller_id': np.max,
              'shipping_limit_date': np.max,
              'price': np.sum, 'freight_value': np.sum,
              'product_description_lenght': np.mean,
              'product_weight_g': np.mean,
              'product_volume_cm3': np.mean,
              'product_category_name': most_freq_cat
              }

cat_cols = dfs['ord_it'].columns[dfs['ord_it'].columns.str.contains('cat_')]
agg_dict_2 = {c: np.sum for c in cat_cols}

# Concatenate the dictionaries
agg_dict = dict(set(agg_dict_1.items()) | set(agg_dict_2.items())) 

In [760]:
agg_dict

{'cat_books': <function numpy.sum>,
 'cat_diy': <function numpy.sum>,
 'cat_electronics': <function numpy.sum>,
 'cat_fashion': <function numpy.sum>,
 'cat_food': <function numpy.sum>,
 'cat_furniture': <function numpy.sum>,
 'cat_home': <function numpy.sum>,
 'cat_hygiene': <function numpy.sum>,
 'cat_leisure': <function numpy.sum>,
 'cat_miscellaneous': <function numpy.sum>,
 'cat_multimedia': <function numpy.sum>,
 'cat_office': <function numpy.sum>,
 'cat_unknown': <function numpy.sum>,
 'freight_value': <function numpy.sum>,
 'order_item_id': <function numpy.amax>,
 'price': <function numpy.sum>,
 'product_category_name': <function __main__.most_freq_cat>,
 'product_description_lenght': <function numpy.mean>,
 'product_volume_cm3': <function numpy.mean>,
 'product_weight_g': <function numpy.mean>,
 'seller_id': <function numpy.amax>,
 'shipping_limit_date': <function numpy.amax>}

In [761]:
# Aggregate the lines of each order
dfs['ord_it'] = dfs['ord_it'].groupby('order_id').agg(agg_dict).reset_index()

# Move the order of the columns (put categories at the end)
dfs['ord_it'] = dfs['ord_it'][move_cat_containing(dfs['ord_it'].columns,
                                                  ['cat_'], 'last')]

# Change the name the 'order_item_id' column
dfs['ord_it'] = dfs['ord_it']\
    .rename(columns={"order_item_id": "order_item_nb",
                     "product_description_lenght": "mean_prod_descr_length",
                    "shipping_limit_date": "max_limit_ship_date",
                     "product_category_name": 'main_prod_categ'})

In [762]:
dfs['ord_it'].sample(3)

,order_id,mean_prod_descr_length,order_item_nb,product_volume_cm3,product_weight_g,seller_id,freight_value,price,max_limit_ship_date,main_prod_categ,cat_books,cat_electronics,cat_diy,cat_leisure,cat_miscellaneous,cat_food,cat_hygiene,cat_unknown,cat_fashion,cat_furniture,cat_office,cat_multimedia,cat_home
33470,56e7d463a96db09e951999dd6e74f136,506.00,1,27000.00,2150.00,ed8cb7b190ceb6067227478e48cf8dde,18.45,49.99,2018-07-30 04:25:26,electronics,0.00,49.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
57210,955d6e7631995ef924924d6ae1144a38,394.00,1,17802.00,1200.00,4a3ca9315b744ce9f8e9374361493884,12.16,93.90,2017-06-22 09:05:21,furniture_decor,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,93.90,0.00,0.00,93.90
68048,b10b53f757455227e8b972cb1faba733,125.00,1,1872.00,300.00,d91fb3b7d041e83b64a00a3edfb37e4f,25.63,10.80,2017-08-23 02:50:22,home_appliances,0.00,10.80,0.00,0.00,10.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.80


In [763]:
print("Number of lines in dfs['ord_it']: ", dfs['ord_it'].shape[0])
print("Number of distinct orders: ", dfs['ord_it']['order_id'].nunique())
print("Total number of items sold: ", dfs['ord_it']['order_item_nb'].sum())

Number of lines in dfs['ord_it']:  98666
Number of distinct orders:  98666
Total number of items sold:  112650


Let's now left merge the orders_it database with sellers data:

In [764]:
dfs['ord_it'] = pd.merge(dfs['ord_it'], dfs['sell'],
                         how='left', on='seller_id')\
                         .drop(columns='geolocation_state')

#### Order payments

In [765]:
dfs['ord_pay'].columns

Index(['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value'], dtype='object')

In [766]:
print("Number of unique orders", dfs['ord_pay'].order_id.nunique())
print("Total number of lines (payments)" , dfs['ord_pay'].shape)

Number of unique orders 99440
Total number of lines (payments) (103886, 5)


In [767]:
print("Payment types are: ", dfs['ord_pay'].payment_type.unique())

Payment types are:  ['credit_card' 'boleto' 'voucher' 'debit_card' 'not_defined']


Payments can be divided into 5 types :
'credit_card', 'boleto', 'voucher', 'debit_card', 'not_defined'

However some payments (same 'order_id') can be made by 2 different means. The number of payments made for each mean is filled in 'payment_sequential'.
NB: 'payment_sequential' is not the same as 'payment_installments', as the latter refers to a scheduling of payments over time.

We'll set a column for each 'payment_type' and we'll fill these columns with the corresponding 'payment_sequential' value. Then we'll aggregate (sum) the lines with the same 'order_id' in order to get one line per order.

In [768]:
# Create dummy columns for payment_type
pay_type_df = pd.get_dummies(dfs['ord_pay']['payment_type'],
                             prefix='paytype')\
                                .mul(dfs['ord_pay']['payment_sequential'],
                                     axis=0)
dfs['ord_pay'] = pd.concat([dfs['ord_pay'], pay_type_df], axis=1)

In [769]:
# Aggregate paytypes columns by order
paytype_cols = ['paytype_boleto', 'paytype_credit_card','paytype_debit_card',
               'paytype_not_defined', 'paytype_voucher','payment_installments',
               'payment_value']
df1 = dfs['ord_pay'].groupby('order_id')[paytype_cols].agg(np.sum)

In [770]:
df1.sample(2)

,paytype_boleto,paytype_credit_card,paytype_debit_card,paytype_not_defined,paytype_voucher,payment_installments,payment_value
order_id,,,,,,,
6ab7569ca759f8bc38432f315743fcd3,0,1,0,0,0,3,160.57
8edaa376e19d08bc84ab8845682216b4,0,1,0,0,0,1,219.27


In [771]:
# Aggregate paytype in a cumulated paytype column
def cumul_paytype(x): return ''.join([s[0].upper() for s in sorted(x.unique())])
df2 = dfs['ord_pay'].groupby('order_id')['payment_type'].agg(cumul_paytype)\
    .reset_index().rename(columns={'payment_type': 'cum_paytype'})

In [772]:
df2.sample(2)

,order_id,cum_paytype
5953,0f3358d8ff329783796c1876bf8aa920,C
19022,313e60572316093cd1a9a5db042389a9,C


In [773]:
# Merge the two aggregated dataframes
dfs['ord_pay'] = pd.merge(df2, df1, on='order_id')

In [774]:
dfs['ord_pay'].sample(6)

,order_id,cum_paytype,paytype_boleto,paytype_credit_card,paytype_debit_card,paytype_not_defined,paytype_voucher,payment_installments,payment_value
95045,f4ab7da19e58e253e0946d32501b279c,C,0,1,0,0,0,2,111.66
64267,a61bafa3b59313a79ecd7be1ca2a2477,C,0,1,0,0,0,6,277.08
59258,9980a5ade25fc8add3023440935ef36d,C,0,1,0,0,0,6,609.24
19723,330534a2e7ad98d0c76ee20bfa001c50,V,0,0,0,0,10,4,63.17
89385,e5b8b27e1436ae108e5e2712a22c0033,B,1,0,0,0,0,1,133.28
73093,bc2e9d05db85d8bdb5e4e7452086e00f,C,0,1,0,0,0,1,107.89


In [775]:
print("Total number of lines (payments) after aggregation",
      dfs['ord_pay'].shape,)

Total number of lines (payments) after aggregation (99440, 9)


#### Order reviews

In [776]:
dfs['ord_rev'].columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp'],
      dtype='object')

We'll then drop all the reviews with same 'review_id' and same 'review_creation_date' (duplicates).

In [777]:
ind_drop = dfs['ord_rev'][dfs['ord_rev'][['review_id',
                                          'review_creation_date']]\
                          .duplicated(keep='first')].index
print("nb of lines to drop: ", ind_drop.shape)
dfs['ord_rev'] = dfs['ord_rev'].drop(index=ind_drop)

nb of lines to drop:  (827,)


Let's create a column that give the length of the message.

In [778]:
dfs['ord_rev']['review_comment_length'] = \
    dfs['ord_rev']['review_comment_message'].str.len()

Now we can drop the columns we won't use.

In [779]:
dfs['ord_rev'] = dfs['ord_rev'].drop(columns=['review_answer_timestamp',
                                              'review_comment_title',
                                              'review_creation_date',
                                              'review_comment_message'])

In [780]:
print("Number of orders: ", dfs['ord_rev'].shape)
print("Number of unique orders: ", dfs['ord_rev']['order_id'].nunique())

Number of orders:  (99173, 4)
Number of unique orders:  98926


Some orders have different items.
Each item may have a different comment and review, however the object of the review (item ?) is unfortunately not mentioned:

In [781]:
dfs['ord_rev'][dfs['ord_rev'].duplicated('order_id', keep=False)]\
                             .sort_values('order_id').head(7)

,review_id,order_id,review_score,review_comment_length
17975,2eab0b2e6f5bded4d9b0b2afcfdf4534,02e0b68852217f5715fb9cc885829454,4,23.00
36081,fa493ead9b093fb0fa6f7d4905b0ef3b,02e0b68852217f5715fb9cc885829454,4,27.00
38672,1dee9d450a2b8533d2085cef898ca4a2,03515a836bb855b03f7df9dee520a8fc,5,nan
50019,5e1ee2e924a1dd10d80b99555383c0a7,03515a836bb855b03f7df9dee520a8fc,5,nan
8329,b04ed893318da5b863e878cd3d0511df,03c939fd7fd3b38f8485a0f95798f1f6,3,180.00
70005,405eb2ea45e1dbe2662541ae5b47e2aa,03c939fd7fd3b38f8485a0f95798f1f6,3,69.00
43434,c793b25686e244588d4fc71964c95a2c,04f1827088d972a62224f5203a071500,1,nan


Lets's then aggregate the reviews relative to the same order.

We'll take the mean of the review score and the mean of the review_comment_length.

In [782]:
dfs['ord_rev'] = \
    dfs['ord_rev'].groupby('order_id')[['review_score',
                                        'review_comment_length']]\
    .mean().reset_index()

In [783]:
dfs['ord_rev'].sample(3)

,order_id,review_score,review_comment_length
84382,da225376046ab771e41a24f84f53f536,5.00,nan
62139,a1991f23b5a551188b16f0cc2105441a,5.00,nan
48553,7dd829472da0fa5d56b7eb94ec5be1b1,5.00,199.00


Let's see if there are missing data.

In [784]:
dfs['ord_rev'].isna().sum()

order_id                     0
review_score                 0
review_comment_length    57562
dtype: int64

We fill the missing review comment length with 0. 

In [785]:
dfs['ord_rev'].review_comment_length = \
    dfs['ord_rev'].review_comment_length.fillna(0)

In [786]:
print("Number of orders: ", dfs['ord_rev'].shape)
print("Number of unique orders: ", dfs['ord_rev']['order_id'].nunique())

Number of orders:  (98926, 3)
Number of unique orders:  98926


#### Orders

In [787]:
dfs['orders'].columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date'],
      dtype='object')

Let's create more appropriate colums relative to shipping time: 

In [788]:
dfs['orders']['shipping_time'] = \
    dfs['orders']['order_delivered_customer_date']\
    -dfs['orders']['order_purchase_timestamp']
dfs['orders']['shipping_delay'] = \
    dfs['orders']['order_delivered_customer_date'] - \
    dfs['orders']['order_estimated_delivery_date'] 
dfs['orders']['delivered'] = \
    dfs['orders']['order_delivered_customer_date'].notna()\
                                                  .map({False: 0, True: 1})

We then drop the columns we won't use.

In [789]:
dfs['orders'] = dfs['orders'].drop(columns=['order_delivered_carrier_date',
                                            'order_approved_at', 
                                            # 'order_purchase_timestamp',
                                            'order_delivered_customer_date',
                                            'order_estimated_delivery_date'])

In [790]:
print("Number of orders: ", dfs['orders'].shape)
print("Number of unique orders: ", dfs['orders']['order_id'].nunique())

Number of orders:  (99441, 7)
Number of unique orders:  99441


In [791]:
dfs['orders'].sample(3)

,order_id,customer_id,order_status,order_purchase_timestamp,shipping_time,shipping_delay,delivered
67272,59bc56d45d476a352332408dfd911694,d0d68a2590a16593ac99c7912b912337,delivered,2018-02-19 11:11:28,18 days 13:47:07,-6 days +00:58:35,1
67665,79f0079f45f137b3a4d31ebbd20de6c2,0078c6b33ae4a3f03de57a8ad36648e1,delivered,2017-04-17 14:31:01,6 days 20:10:59,-16 days +10:42:00,1
33422,0e48ca107de4291a6f7536b4f5cf512e,282fbce48e4d2077aad602dd125c9225,delivered,2018-05-15 10:39:41,7 days 09:02:31,-21 days +19:42:12,1


#### Merging orders' data

We'll merge all the data from:
- orders database (99 441 unique orders)
- order items database (98 666 unique orders)
- order payments database (99 440 unique orders)
- order review database
in the orders database.

In [792]:
for name_df in ['orders', 'ord_it', 'ord_rev', 'ord_pay']:
    print("---------", name_df, dfs[name_df].shape)
    print(dfs[name_df].columns)

--------- orders (99441, 7)
Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'shipping_time',
       'shipping_delay', 'delivered'],
      dtype='object')
--------- ord_it (98666, 26)
Index(['order_id', 'mean_prod_descr_length', 'order_item_nb', 'product_volume_cm3',
       'product_weight_g', 'seller_id', 'freight_value', 'price', 'max_limit_ship_date',
       'main_prod_categ', 'cat_books', 'cat_electronics', 'cat_diy', 'cat_leisure',
       'cat_miscellaneous', 'cat_food', 'cat_hygiene', 'cat_unknown', 'cat_fashion',
       'cat_furniture', 'cat_office', 'cat_multimedia', 'cat_home', 'seller_lat', 'seller_long',
       'seller_region'],
      dtype='object')
--------- ord_rev (98926, 3)
Index(['order_id', 'review_score', 'review_comment_length'], dtype='object')
--------- ord_pay (99440, 9)
Index(['order_id', 'cum_paytype', 'paytype_boleto', 'paytype_credit_card', 'paytype_debit_card',
       'paytype_not_defined', 'paytype_voucher', 'payment_installment

Some orders may not have 'items'(order_status: canceled or unavailable),  'payments' nor 'reviews', that is why distinct orders in df['order'] are 99441 and in df['ord_it'] and df['ord_rev'] are only 98666 and resp. 98926.

We want to keep all the orders and will merge df['order'] to df['ord_it'], df['ord_rev']and ['ord_pay'] with a left join, as we will drop orders that has no payment, no review or no item.

In [793]:
# left join because item, payment or review data are not useful
# without data from the orders database (customer_id, status ...) 
dfs['orders'] = pd.merge(dfs['orders'], dfs['ord_it'],
                         how='left', on='order_id')
dfs['orders'] = pd.merge(dfs['orders'], dfs['ord_rev'],
                         how='left', on='order_id')
dfs['orders'] = pd.merge(dfs['orders'], dfs['ord_pay'],
                         how='left', on='order_id')

Let's see if our merged dataframe has missing data, and if a filling strategy can be applied.

In [794]:
dfs['orders'].isna().sum()

order_id                       0
customer_id                    0
order_status                   0
order_purchase_timestamp       0
shipping_time               2965
shipping_delay              2965
delivered                      0
mean_prod_descr_length       775
order_item_nb                775
product_volume_cm3           775
product_weight_g             775
seller_id                    775
freight_value                775
price                        775
max_limit_ship_date          775
main_prod_categ              775
cat_books                    775
cat_electronics              775
cat_diy                      775
cat_leisure                  775
cat_miscellaneous            775
cat_food                     775
cat_hygiene                  775
cat_unknown                  775
cat_fashion                  775
cat_furniture                775
cat_office                   775
cat_multimedia               775
cat_home                     775
seller_lat                   992
seller_lon

Let's see why 'shipping_time' and 'shipping_delay' may be null:

In [795]:
dfs['orders'][dfs['orders']['shipping_time'].isna()].sample(5)

,order_id,customer_id,order_status,order_purchase_timestamp,shipping_time,shipping_delay,delivered,mean_prod_descr_length,order_item_nb,product_volume_cm3,product_weight_g,seller_id,freight_value,price,max_limit_ship_date,main_prod_categ,cat_books,cat_electronics,cat_diy,cat_leisure,cat_miscellaneous,cat_food,cat_hygiene,cat_unknown,cat_fashion,cat_furniture,cat_office,cat_multimedia,cat_home,seller_lat,seller_long,seller_region,review_score,review_comment_length,cum_paytype,paytype_boleto,paytype_credit_card,paytype_debit_card,paytype_not_defined,paytype_voucher,payment_installments,payment_value
85901,b102b42cde7719ae4b7c0163cf4d7dbe,6df75bdce4569326dcfdf7fc9bbe5981,canceled,2018-08-29 10:15:27,NaT,NaT,0,nan,nan,nan,nan,NaN,nan,nan,NaT,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,3.00,41.00,V,0.00,0.00,0.00,0.00,1.00,1.00,468.11
10110,52c1dd5de2f994e2979bb421330763ad,fc6970358133b4ddd73d4fe57cfe6737,shipped,2017-11-24 19:36:27,NaT,NaT,0,774.00,1.00,21460.00,2633.00,46dc3b2cc0980fb8ec44634e21d2718e,22.45,229.99,2017-11-29 22:55:04,toys,0.00,0.00,0.00,229.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-22.93,-43.18,SE,1.00,192.00,C,0.00,1.00,0.00,0.00,0.00,10.00,252.44
55780,bcd03a007dcad04bc88701f563b985b7,39f707c5c87d5f8c8e4a26ce95292a81,shipped,2017-02-08 21:31:38,NaT,NaT,0,411.00,1.00,1600.00,150.00,cab85505710c7cb9b720bceb52b01cee,8.72,49.90,2017-02-12 21:31:38,fashion_bags_accessories,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,49.90,0.00,0.00,0.00,0.00,-23.48,-46.59,SE,5.00,0.00,C,0.00,1.00,0.00,0.00,0.00,1.00,58.62
80587,67384817c871ae183b24f9a0a6eb9bbc,f83083fcbc51d12f8279b5713c2d4b4d,shipped,2018-01-25 12:12:37,NaT,NaT,0,666.00,1.00,3380.00,700.00,c66dccfb3f109511246da627dd5a2498,28.96,526.00,2018-01-31 12:31:27,watches_gifts,0.00,526.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-24.41,-47.74,SE,1.00,0.00,C,0.00,1.00,0.00,0.00,0.00,8.00,554.96
92446,de779f93dc5f7dd9ac51b22380685ae5,4627f0d16c02256e8bd28f9bbb1ee468,shipped,2018-06-02 10:57:17,NaT,NaT,0,128.00,1.00,12500.00,950.00,7ad32824caee82087b3e2e5f33b1bf32,19.32,50.00,2018-06-13 04:30:33,bed_bath_table,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,50.00,-21.73,-48.81,SE,1.00,22.00,B,1.00,0.00,0.00,0.00,0.00,1.00,69.32


It is only when articles are not yet delivered:

In [796]:
dfs['orders'][dfs['orders']['shipping_time'].isna()\
              & dfs['orders']['delivered']==True]

,order_id,customer_id,order_status,order_purchase_timestamp,shipping_time,shipping_delay,delivered,mean_prod_descr_length,order_item_nb,product_volume_cm3,product_weight_g,seller_id,freight_value,price,max_limit_ship_date,main_prod_categ,cat_books,cat_electronics,cat_diy,cat_leisure,cat_miscellaneous,cat_food,cat_hygiene,cat_unknown,cat_fashion,cat_furniture,cat_office,cat_multimedia,cat_home,seller_lat,seller_long,seller_region,review_score,review_comment_length,cum_paytype,paytype_boleto,paytype_credit_card,paytype_debit_card,paytype_not_defined,paytype_voucher,payment_installments,payment_value


In [797]:
dfs['orders'][dfs['orders']['shipping_time'].isna()]['order_status'].unique()

array(['invoiced', 'shipped', 'processing', 'unavailable', 'canceled',
       'delivered', 'created', 'approved'], dtype=object)

#### Filling missing data in Orders dataframe

Let's see if there are missing data:

In [798]:
dfs['orders'].isna().sum()

order_id                       0
customer_id                    0
order_status                   0
order_purchase_timestamp       0
shipping_time               2965
shipping_delay              2965
delivered                      0
mean_prod_descr_length       775
order_item_nb                775
product_volume_cm3           775
product_weight_g             775
seller_id                    775
freight_value                775
price                        775
max_limit_ship_date          775
main_prod_categ              775
cat_books                    775
cat_electronics              775
cat_diy                      775
cat_leisure                  775
cat_miscellaneous            775
cat_food                     775
cat_hygiene                  775
cat_unknown                  775
cat_fashion                  775
cat_furniture                775
cat_office                   775
cat_multimedia               775
cat_home                     775
seller_lat                   992
seller_lon

Filling missing values.

In [799]:
# Filling missing dates
dfs['orders']['max_limit_ship_date'] = \
    dfs['orders']['max_limit_ship_date'].fillna(pd.to_datetime('2020-01-01'))

# Filling missing main product category
dfs['orders']['main_prod_categ'] = \
    dfs['orders']['main_prod_categ'].fillna('unknown')

# Filling missing Timedeltas
dfs['orders']['shipping_time'] = \
    dfs['orders']['shipping_time'].fillna(pd.Timedelta(0))
dfs['orders']['shipping_delay'] = \
    dfs['orders']['shipping_delay'].fillna(pd.Timedelta(0))

# Filling missing numerics
cols_to_fill_to_0 = dfs['orders'].isna().sum()[(dfs['orders'].isna().sum()!=0)].index
dfs['orders'][cols_to_fill_to_0] = dfs['orders'][cols_to_fill_to_0].fillna(0)

In [800]:
dfs['orders'].isna().sum()

order_id                    0
customer_id                 0
order_status                0
order_purchase_timestamp    0
shipping_time               0
shipping_delay              0
delivered                   0
mean_prod_descr_length      0
order_item_nb               0
product_volume_cm3          0
product_weight_g            0
seller_id                   0
freight_value               0
price                       0
max_limit_ship_date         0
main_prod_categ             0
cat_books                   0
cat_electronics             0
cat_diy                     0
cat_leisure                 0
cat_miscellaneous           0
cat_food                    0
cat_hygiene                 0
cat_unknown                 0
cat_fashion                 0
cat_furniture               0
cat_office                  0
cat_multimedia              0
cat_home                    0
seller_lat                  0
seller_long                 0
seller_region               0
review_score                0
review_com

#### Customers

We see here that each 'customer_id' of the df['orders'] dataset correspond to one unique order.

In [801]:
dfs['orders']['customer_id'].nunique(), dfs['orders']['order_id'].nunique()

(99441, 99441)

The customer_id that correspond to one unique customer (one person) can be found in the df['cust'] dataset.

In [802]:
dfs['cust']['customer_id'].nunique(), dfs['cust']['customer_unique_id'].nunique()

(99441, 96096)

In [803]:
dfs['cust'] = pd.merge(dfs['cust'], dfs['geoloc'], how='left',
                       left_on='customer_zip_code_prefix',
                       right_on='geolocation_zip_code_prefix')
dfs['cust'] = dfs['cust'].drop(columns=['customer_state', 'customer_city',
                                        'customer_zip_code_prefix',
                                        'geolocation_zip_code_prefix',
                                        'geolocation_state'])\
                         .rename(columns={'region': 'cust_region',
                                          'geolocation_lat': 'cust_lat',
                                          'geolocation_lng': 'cust_long'})

Let's merge the dfs['cust'] and dfs['orders'] dataset, and drop the 'customer_id' column to get a big dataset of all the orders.

In [804]:
# left join (customers data without order_id and customer_id is not useful)
dfs['orders'] = pd.merge(dfs['orders'], dfs['cust'],
                         how='left', on='customer_id')
dfs['orders'] = dfs['orders'].drop(columns=['customer_id'])

In [805]:
dfs['orders'].columns

Index(['order_id', 'order_status', 'order_purchase_timestamp', 'shipping_time', 'shipping_delay',
       'delivered', 'mean_prod_descr_length', 'order_item_nb', 'product_volume_cm3',
       'product_weight_g', 'seller_id', 'freight_value', 'price', 'max_limit_ship_date',
       'main_prod_categ', 'cat_books', 'cat_electronics', 'cat_diy', 'cat_leisure',
       'cat_miscellaneous', 'cat_food', 'cat_hygiene', 'cat_unknown', 'cat_fashion',
       'cat_furniture', 'cat_office', 'cat_multimedia', 'cat_home', 'seller_lat', 'seller_long',
       'seller_region', 'review_score', 'review_comment_length', 'cum_paytype', 'paytype_boleto',
       'paytype_credit_card', 'paytype_debit_card', 'paytype_not_defined', 'paytype_voucher',
       'payment_installments', 'payment_value', 'customer_unique_id', 'cust_lat', 'cust_long',
       'cust_region'],
      dtype='object')

Let's now reorder the columns:

In [806]:
ordered_cols = move_cat_containing(dfs['orders'].columns, ['pay'], 'last')
ordered_cols = move_cat_containing(ordered_cols, ['cat'], 'last')
ordered_cols = move_cat_containing(ordered_cols, ['ship'], 'first')
ordered_cols = move_cat_containing(ordered_cols, ['prod'], 'first')
ordered_cols = move_cat_containing(ordered_cols, ['order'], 'first')
ordered_cols = move_cat_containing(ordered_cols, ['cust'], 'first')
dfs['orders'] = dfs['orders'][ordered_cols]

In [807]:
dfs['orders'].head(3)

,customer_unique_id,cust_lat,cust_long,cust_region,order_id,order_status,order_purchase_timestamp,order_item_nb,mean_prod_descr_length,product_volume_cm3,product_weight_g,main_prod_categ,shipping_time,shipping_delay,max_limit_ship_date,delivered,seller_id,freight_value,price,seller_lat,seller_long,seller_region,review_score,review_comment_length,cum_paytype,paytype_boleto,paytype_credit_card,paytype_debit_card,paytype_not_defined,paytype_voucher,payment_installments,payment_value,cat_books,cat_electronics,cat_diy,cat_leisure,cat_miscellaneous,cat_food,cat_hygiene,cat_unknown,cat_fashion,cat_furniture,cat_office,cat_multimedia,cat_home
0,7c396fd4830fd04220f754e42b4e5bff,-23.57,-46.59,SE,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,1.00,268.00,1976.00,500.00,housewares,8 days 10:28:40,-8 days +21:25:13,2017-10-06 11:07:15,1,3504c0cb71d7fa48d967e0e4c94d59d9,8.72,29.99,-23.67,-46.44,SE,4.00,170.00,CV,0.00,1.00,0.00,0.00,5.00,3.00,38.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,29.99
1,af07308b275d755c9edb36a90c618231,-12.12,-38.46,NE,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,1.00,178.00,4693.00,400.00,perfumery,13 days 18:46:08,-6 days +15:27:45,2018-07-30 03:24:27,1,289cdb325fb7e7f891c38608bf9e0962,22.76,118.70,-19.80,-43.97,SE,4.00,20.00,B,1.00,0.00,0.00,0.00,0.00,1.00,141.46,0.00,0.00,0.00,0.00,0.00,0.00,118.70,0.00,0.00,0.00,0.00,0.00,0.00
2,3a653a41f6f9fc3d2a113cf8398680e8,-16.74,-48.50,CW,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,1.00,232.00,9576.00,420.00,auto,9 days 09:27:40,-18 days +18:06:29,2018-08-13 08:55:23,1,4869f7a5dfa277a7dca6462dcf3b52b2,19.22,159.90,-21.35,-48.22,SE,5.00,0.00,C,0.00,1.00,0.00,0.00,0.00,3.00,179.12,0.00,0.00,0.00,0.00,159.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [808]:
dfs['orders'].isna().sum()

customer_unique_id            0
cust_lat                    278
cust_long                   278
cust_region                 278
order_id                      0
order_status                  0
order_purchase_timestamp      0
order_item_nb                 0
mean_prod_descr_length        0
product_volume_cm3            0
product_weight_g              0
main_prod_categ               0
shipping_time                 0
shipping_delay                0
max_limit_ship_date           0
delivered                     0
seller_id                     0
freight_value                 0
price                         0
seller_lat                    0
seller_long                   0
seller_region                 0
review_score                  0
review_comment_length         0
cum_paytype                   0
paytype_boleto                0
paytype_credit_card           0
paytype_debit_card            0
paytype_not_defined           0
paytype_voucher               0
payment_installments          0
payment_

In [809]:
dfs['orders'] = dfs['orders'].fillna(dfs['orders'].mode().iloc[0])

### 0.4 Exportation of the Orders and Orders_items dataset

Now we export the dataset of aggregated orders in a .csv file.

In [810]:
dfs['ord_it'].to_csv('agg_order_items.csv')
dfs['orders'].to_csv('agg_orders.csv')